In [10]:
import os
import pandas as pd
from typing import List, Dict, Optional

def load_local_price_data(
    tickers: List[str],
    dir_path: str = r"D:\shawn_workspace\REAL LAB\REALLAB_FinLLM\Data\history_price_data",
    start: Optional[str] = None,
    end: Optional[str] = None,
) -> Dict[str, pd.DataFrame]:
    price_data_dict = {}

    for ticker in tickers:
        file_path = os.path.join(dir_path, f"{ticker}.csv")
        if not os.path.isfile(file_path):
            raise FileNotFoundError(f"No file:{file_path}")

        df = pd.read_csv(file_path)

        # 统一列名大小写并去空格
        df.columns = df.columns.str.strip().str.title()
        if "Date" not in df.columns:
            raise ValueError(f"{file_path} No 'Date' column")

        # 解析日期 → Datetime；再设为索引
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df.dropna(subset=["Date"]).set_index("Date")

        # **确保索引是 DatetimeIndex**
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index, errors="coerce")
            df = df[~df.index.isna()]

        # 去掉时区（若有）
        if isinstance(df.index, pd.DatetimeIndex) and df.index.tz is not None:
            df.index = df.index.tz_convert(None)

        # 日期过滤
        if start:
            df = df.loc[df.index >= pd.to_datetime(start)]
        if end:
            df = df.loc[df.index <= pd.to_datetime(end)]

        # 只保留常见列
        keep_cols = ["Open", "High", "Low", "Close", "Volume"]
        df = df[[c for c in keep_cols if c in df.columns]]

        price_data_dict[ticker] = df

    return price_data_dict


In [41]:
def load_local_price_data(
    tickers: List[str],
    dir_path: str = r"D:\shawn_workspace\REAL LAB\REALLAB_FinLLM\Data\history_price_data",
    start: Optional[str] = None,
    end: Optional[str] = None,
) -> Dict[str, pd.DataFrame]:
    price_data_dict = {}

    for ticker in tickers:
        file_path = os.path.join(dir_path, f"{ticker}.csv")
        if not os.path.isfile(file_path):
            raise FileNotFoundError(f"No file: {file_path}")

        df = pd.read_csv(file_path)

        # 清洗列名
        df.columns = df.columns.str.strip().str.title()
        if "Date" not in df.columns:
            raise ValueError(f"{file_path} missing 'Date' column")

        # 转换为 datetime（即便带时区也统一），再去掉时区
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce", utc=True)  # 添加 utc 统一化
        df = df.dropna(subset=["Date"])
        df["Date"] = df["Date"].dt.tz_convert(None)  # 去除时区
        df.set_index("Date", inplace=True)

        # 时间过滤
        if start:
            df = df.loc[df.index >= pd.to_datetime(start)]
        if end:
            df = df.loc[df.index <= pd.to_datetime(end)]

        # 保留常用列
        keep_cols = ["Open", "High", "Low", "Close", "Volume"]
        df = df[[col for col in keep_cols if col in df.columns]]

        price_data_dict[ticker] = df

    return price_data_dict


In [42]:
# ① 读取本地历史行情
price_data = load_local_price_data(
    tickers=[ "AMZN"],
    start="2024-01-01",
    end="2024-03-29"
)

# ② 构造你的 position_dict & market_value_dict（示例）
print(price_data)

{'AMZN':                            Open        High         Low       Close    Volume
Date                                                                         
2024-01-02 05:00:00  151.539993  152.380005  148.389999  149.929993  47339400
2024-01-03 05:00:00  149.199997  151.050003  148.330002  148.470001  49425500
2024-01-04 05:00:00  145.589996  147.380005  144.050003  144.570007  56039800
2024-01-05 05:00:00  144.690002  146.589996  144.529999  145.240005  45124800
2024-01-08 05:00:00  146.740005  149.399994  146.149994  149.100006  46757100
...                         ...         ...         ...         ...       ...
2024-03-22 04:00:00  177.750000  179.259995  176.750000  178.869995  27964100
2024-03-25 04:00:00  178.009995  180.990005  177.240005  179.710007  29815500
2024-03-26 04:00:00  180.149994  180.449997  177.949997  178.300003  29659000
2024-03-27 04:00:00  179.880005  180.000000  177.309998  179.830002  33272600
2024-03-28 04:00:00  180.169998  181.699997  179.259995

In [43]:
print(len(price_data["AMZN"]))

61


In [16]:
price_data["AAPL"].iloc[0]['Open']

185.78943757371047

In [14]:
import yfinance as yf

data = yf.download("TSLA", start="2024-01-25", end="2024-01-30")

print(type(data))
print(data.head(2))


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


<class 'pandas.core.frame.DataFrame'>
Empty DataFrame
Columns: [(Adj Close, TSLA), (Close, TSLA), (High, TSLA), (Low, TSLA), (Open, TSLA), (Volume, TSLA)]
Index: []


In [ ]:
import yfinance as yf

data = yf.download("TSLA", start="2024-01-01", end="2024-12-30")

print(type(data))
print(data.head(2))


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


<class 'pandas.core.frame.DataFrame'>
Empty DataFrame
Columns: [(Adj Close, TSLA), (Close, TSLA), (High, TSLA), (Low, TSLA), (Open, TSLA), (Volume, TSLA)]
Index: []


In [2]:
# ✅ 直接 await 即可
from data_collection.alvan_dc.historical_price import fetch_single_adjdaily
result = await fetch_single_adjdaily("TSLA")
print(result)


{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}
[Error] Unexpected response: {'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}
{'TSLA': None}


In [4]:
import pandas as pd

def evaluate_trading_strategy(agent_outputs: list[dict], price_df: pd.DataFrame, initial_cash=10000.0):
    """
    Simulate trading over time and evaluate profit/loss.
    
    Args:
        agent_outputs (list[dict]): 每天一个agent决策字典，如：
            [{'date': '2024-01-25', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'}, ...]
        price_df (pd.DataFrame): yfinance下载的价格，index为日期，包含 'Open', 'Close' 列
        initial_cash (float): 初始本金

    Returns:
        final_cash (float): 模拟结束时账户价值（含现金+股票）
        daily_log (list[dict]): 每天的持仓、行为、现金等记录
    """
    position = 'FLAT'
    shares_held = 0
    cash = initial_cash
    daily_log = []

    # 确保时间按顺序
    agent_outputs = sorted(agent_outputs, key=lambda x: x["date"])
    print(len(agent_outputs))
    for i in range(len(agent_outputs)):
        today = agent_outputs[i]["date"]
        today_action = agent_outputs[i]["trader"]
        executed = agent_outputs[i]["manager"] == "EXECUTE_TRADE"
        today_dt = pd.to_datetime(today)

        # 取次日开盘价作为交易价格（如果可得）
        try:
            next_day_price = price_df.loc[today_dt + pd.Timedelta(days=1)]["Open"]
        except KeyError:
            break  # 没有次日价格，无法再交易

        # 记录前状态
        record = {
            "date": today,
            "position_before": position,
            "action": today_action,
            "executed": executed,
            "price": next_day_price,
            "shares_held_before": shares_held,
            "cash_before": cash,
        }

        # 交易逻辑
        if executed:
            if today_action == "BUY" and position == "FLAT":
                shares_held = int(cash // next_day_price)
                cash -= shares_held * next_day_price
                position = "LONG"
                record["real_action"] = "BUY_EXECUTED"
            elif today_action == "SELL" and position == "LONG":
                cash += shares_held * next_day_price
                shares_held = 0
                position = "FLAT"
                record["real_action"] = "SELL_EXECUTED"
            else:
                record["real_action"] = "NO_OP"  # HOLD 或空仓重复SELL
        else:
            record["real_action"] = "NO_OP"

        # 计算当日总资产
        if position == "LONG":
            today_price = price_df.loc[today_dt]["Close"]
            record["asset_value"] = cash + shares_held * today_price
        else:
            record["asset_value"] = cash

        record["position_after"] = position
        record["shares_held_after"] = shares_held
        record["cash_after"] = cash
        print("logging record:")
        print(record)
        daily_log.append(record)
        print(len(daily_log))
    # 最后一天按持仓估值
    final_value = daily_log[-1]["asset_value"] if daily_log else initial_cash
    return final_value, daily_log


agent_outputs = [
    {'date': '2024-01-25', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-26', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'DO_NOT_EXECUTE'},
    {'date': '2024-01-27', 'trader': 'SELL', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-28', 'trader': 'SELL', 'risk': 'APPROVED', 'manager': 'DO_NOT_EXECUTE'},
    {'date': '2024-01-29', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
]

price_df = yf.download("TSLA", start="2024-01-25", end="2024-01-31")

final_value, log = evaluate_trading_strategy(agent_outputs, price_df)


[*********************100%***********************]  1 of 1 completed

5
logging record:
{'date': '2024-01-25', 'position_before': 'FLAT', 'action': 'BUY', 'executed': True, 'price': Ticker
TSLA    185.5
Name: 2024-01-26 00:00:00, dtype: float64, 'shares_held_before': 0, 'cash_before': 10000.0, 'real_action': 'BUY_EXECUTED', 'asset_value': Ticker
TSLA    9847.890259
dtype: float64, 'position_after': 'LONG', 'shares_held_after': 53, 'cash_after': Ticker
TSLA    168.5
Name: 2024-01-26 00:00:00, dtype: float64}
1



C:\Users\Spencer\AppData\Local\Temp\ipykernel_1256\2620667896.py:51: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  shares_held = int(cash // next_day_price)


In [22]:
agent_outputs = [
    {'date': '2024-01-25', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-26', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'DO_NOT_EXECUTE'},
    {'date': '2024-01-27', 'trader': 'SELL', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-28', 'trader': 'SELL', 'risk': 'APPROVED', 'manager': 'DO_NOT_EXECUTE'},
    {'date': '2024-01-29', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
]

price_df = yf.download("TSLA", start="2024-01-25", end="2024-01-31")

final_value, log = evaluate_trading_strategy(agent_outputs, price_df)

from pprint import pprint
pprint(log)
print("Final portfolio value:", final_value)


[*********************100%***********************]  1 of 1 completed

5
{'date': '2024-01-25', 'position_before': 'FLAT', 'action': 'BUY', 'executed': True, 'price': Ticker
TSLA    185.5
Name: 2024-01-26 00:00:00, dtype: float64, 'shares_held_before': 0, 'cash_before': 10000.0, 'real_action': 'BUY_EXECUTED', 'asset_value': Ticker
TSLA    9847.890259
dtype: float64, 'position_after': 'LONG', 'shares_held_after': 53, 'cash_after': Ticker
TSLA    168.5
Name: 2024-01-26 00:00:00, dtype: float64}
1
[{'action': 'BUY',
  'asset_value': Ticker
TSLA    9847.890259
dtype: float64,
  'cash_after': Ticker
TSLA    168.5
Name: 2024-01-26 00:00:00, dtype: float64,
  'cash_before': 10000.0,
  'date': '2024-01-25',
  'executed': True,
  'position_after': 'LONG',
  'position_before': 'FLAT',
  'price': Ticker
TSLA    185.5
Name: 2024-01-26 00:00:00, dtype: float64,
  'real_action': 'BUY_EXECUTED',
  'shares_held_after': 53,
  'shares_held_before': 0}]
Final portfolio value: Ticker
TSLA    9847.890259
dtype: float64



C:\Users\Spencer\AppData\Local\Temp\ipykernel_4944\3065201703.py:51: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  shares_held = int(cash // next_day_price)


In [ ]:
def get_position_list(agent_outputs: list[dict]) -> list[int]:
    """
    Given a list of agent decisions, return a list of 0/1 indicating holding status after each day.

    Args:
        agent_outputs (list[dict]): List of daily agent outputs. Each item must include:
            - 'date': str
            - 'trader': 'BUY' or 'SELL'
            - 'manager': 'EXECUTE_TRADE' or 'DO_NOT_EXECUTE'

    Returns:
        list[int]: 0 for no position (FLAT), 1 for holding (LONG), aligned with agent_outputs
    """
    position = 0  # 0: FLAT, 1: LONG
    position_list = []

    for output in agent_outputs:
        action = output["trader"]
        executed = output["manager"] == "EXECUTE_TRADE"

        if executed:
            if action == "BUY":
                position = 1
            elif action == "SELL":
                position = 0

        position_list.append(position)

    return position_list


In [ ]:
agent_outputs = [
    {'date': '2024-01-25', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-26', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'DO_NOT_EXECUTE'},
    {'date': '2024-01-27', 'trader': 'SELL', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-28', 'trader': 'SELL', 'risk': 'APPROVED', 'manager': 'DO_NOT_EXECUTE'},
    {'date': '2024-01-29', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-29', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-27', 'trader': 'SELL', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
    {'date': '2024-01-29', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'DO_NOT_EXECUTE'},
    {'date': '2024-01-29', 'trader': 'BUY', 'risk': 'APPROVED', 'manager': 'EXECUTE_TRADE'},
]

position_list = get_position_list(agent_outputs)
print(position_list)  # [1, 1, 0, 0, 1, 1, 0, 0, 1]

# position_dict:  {'AMZN': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


[1, 1, 0, 0, 1, 1, 0, 0, 1]


In [108]:
import pprint

a=[{'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-02'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-03'}, {'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-01-04'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-05'}, {'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-01-08'}, {'trader': None, 'risk': 'REJECTED', 'risk_tag': None, 'manager': None, 'date': '2024-01-09'}, {'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-01-10'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-11'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-12'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-16'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-17'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-18'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-19'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-22'}, {'trader': None, 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-23'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-24'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-25'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-26'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-29'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-30'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-01-31'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-01'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-02'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-05'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-06'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-07'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-08'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-09'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'DO_NOT_EXECUTE', 'date': '2024-02-12'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-13'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-14'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-15'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-16'}, {'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-02-20'}, {'trader': None, 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-21'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-22'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-23'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-26'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-27'}, {'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-02-28'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-02-29'}, {'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-03-01'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': None, 'manager': 'EXECUTE_TRADE', 'date': '2024-03-04'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-05'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-06'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': None, 'date': '2024-03-07'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-08'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-11'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-12'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-13'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-14'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-15'}, {'trader': 'SELL', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-03-18'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-19'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': None, 'date': '2024-03-20'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-21'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-22'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-25'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-26'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-27'}, {'trader': 'SELL', 'risk': 'APPROVED', 'risk_tag': 'Neutral', 'manager': 'EXECUTE_TRADE', 'date': '2024-03-28'}]
# pprint.pprint(a)
print(len(a))
for aa in a:
    if aa['trader'] == 'BUY':
        print(aa)

61
{'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-01-04'}
{'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-01-08'}
{'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-01-10'}
{'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-02-20'}
{'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-02-28'}
{'trader': 'BUY', 'risk': 'REJECTED', 'risk_tag': None, 'manager': 'DO_NOT_EXECUTE', 'date': '2024-03-01'}


In [23]:
res = await get_stock_price_history('AMZN', '2024-01-25', 'full')
print(res)


Data available from 1999-11-01 to 2025-05-02
{'AMZN': {'2024-01-25': {'1. open': '156.95', '2. high': '158.505', '3. low': '154.5501', '4. close': '157.75', '5. adjusted close': '157.75', '6. volume': '43638592', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2024-01-24': {'1. open': '157.8', '2. high': '158.51', '3. low': '156.48', '4. close': '156.87', '5. adjusted close': '156.87', '6. volume': '47636725', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2024-01-23': {'1. open': '154.85', '2. high': '156.21', '3. low': '153.93', '4. close': '156.02', '5. adjusted close': '156.02', '6. volume': '37986039', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2024-01-22': {'1. open': '156.89', '2. high': '157.05', '3. low': '153.9', '4. close': '154.78', '5. adjusted close': '154.78', '6. volume': '43687468', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2024-01-19': {'1. open': '153.83', '2. high': '155.76', '3. low': '

In [5]:
res = await get_stock_price_history('AMZN', '2024-01-25', 'full')
print(res)


Data available from 1999-11-01 to 2025-05-02
{'AMZN': {'2024-01-25': {'adjusted close': '157.75', 'volume': '43638592'}, '2024-01-24': {'adjusted close': '156.87', 'volume': '47636725'}, '2024-01-23': {'adjusted close': '156.02', 'volume': '37986039'}, '2024-01-22': {'adjusted close': '154.78', 'volume': '43687468'}, '2024-01-19': {'adjusted close': '155.34', 'volume': '51651628'}, '2024-01-18': {'adjusted close': '153.5', 'volume': '37850245'}, '2024-01-17': {'adjusted close': '151.71', 'volume': '34953363'}, '2024-01-16': {'adjusted close': '153.16', 'volume': '41384636'}, '2024-01-12': {'adjusted close': '154.62', 'volume': '40484155'}, '2024-01-11': {'adjusted close': '155.18', 'volume': '49072691'}, '2024-01-10': {'adjusted close': '153.73', 'volume': '44421830'}, '2024-01-09': {'adjusted close': '151.37', 'volume': '43812567'}, '2024-01-08': {'adjusted close': '149.1', 'volume': '46757053'}, '2024-01-05': {'adjusted close': '145.24', 'volume': '44770343'}, '2024-01-04': {'adjuste

In [2]:
import pandas as pd

def evaluate_portfolio_performance(
    position_dict: dict,
    market_value_dict: dict,
    price_data_dict: dict,
    initial_cash: float = 10000.0
):
    """
    Evaluate portfolio performance with daily rebalancing based on position signals and market values.

    Args:
        position_dict (dict): {ticker: [0/1, 0/1, ...]} for each day
        market_value_dict (dict): {ticker: [mv1, mv2, ...]} for each day
        price_data_dict (dict): {ticker: pd.DataFrame with 'Open' and 'Close', index is datetime}
        initial_cash (float): Starting cash

    Returns:
        dict with:
            - daily_values: list of portfolio values per day
            - CR: cumulative return
            - AR: annualized return
    """
    tickers = position_dict.keys()
    n_days = len(next(iter(position_dict.values())))
    cash = initial_cash
    holdings = {ticker: 0.0 for ticker in tickers}
    daily_values = []
    previous_active_tickers = None
    change_portfolio = False

    for day_idx in range(n_days):
        print("\nnew day")
        active_tickers = [t for t in tickers if position_dict[t][day_idx] == 1]
        # 1. Sell all previous positions at today's open price
        # if  previous_active_tickers==active_tickers: 不用卖
        if not previous_active_tickers or previous_active_tickers != active_tickers: # 有变动，要买股票重组
            change_portfolio = True
            print("有变动")
            for ticker in tickers:
                if float(holdings[ticker]) > 0:
                    open_price = float(price_data_dict[ticker].iloc[day_idx]['Open'])
                    cash += holdings[ticker] * open_price
                    holdings[ticker] = 0.0
            print("变动前 卖掉所有 ", holdings)
        else:
            change_portfolio = False
        print("cash: ", cash)
        # 2. Determine which tickers to long today
        
        previous_active_tickers = active_tickers
        
        if active_tickers and change_portfolio:
            print("有股票要买")
            total_mv = sum(market_value_dict[t][day_idx] for t in active_tickers)

            for ticker in active_tickers:
                weight = market_value_dict[ticker][day_idx] / total_mv
                allocation = cash * weight
                open_price = float(price_data_dict[ticker].iloc[day_idx]['Open'])
                shares = allocation / open_price
                print(ticker," weight ",weight," allocation ",allocation, " shares", shares)
                holdings[ticker] = shares
            cash = 0.0  # all cash invested
            print("买完股票 ", holdings)

        # 3. At end of day, calculate portfolio value using close prices
        total_value = cash
        for ticker in tickers:
            if float(holdings[ticker]) > 0:
                close_price = float(price_data_dict[ticker].iloc[day_idx]['Close'])
                total_value += holdings[ticker] * close_price
        daily_values.append(total_value)
        print(holdings)


    # 4. Compute CR and AR
    CR = daily_values[-1] / initial_cash - 1
    AR = (1 + CR) ** (252 / n_days) - 1
    print("daily_values:", daily_values)
    return {
        "daily_values": daily_values,
        "CR": CR,
        "AR": AR
    }



In [4]:
import os
import pandas as pd

def load_market_value_dict(ticker_list, folder_path):
    """
    Given a list of stock tickers, load their market cap values from CSV files.

    Args:
        ticker_list (list): List of stock tickers, e.g., ["AAPL", "TSLA"]
        folder_path (str): Folder path containing CSV files like AAPL.csv, TSLA.csv, etc.

    Returns:
        dict: {ticker: [market_cap_1, market_cap_2, ...]}
    """
    market_value_dict = {}

    for ticker in ticker_list:
        file_path = os.path.join(folder_path, f"{ticker}.csv")
        try:
            df = pd.read_csv(file_path)
            if 'MarketCap' in df.columns:
                market_value_dict[ticker] = df['MarketCap'].tolist()
            else:
                print(f"Warning: 'MarketCap' column missing in {ticker}.csv")
        except FileNotFoundError:
            print(f"File not found for ticker: {ticker}")
        except Exception as e:
            print(f"Error reading {ticker}.csv: {e}")

    return market_value_dict

tickers = ["AAPL", "TSLA", "AMZN"]
folder = r"D:\shawn_workspace\REAL LAB\REALLAB_FinLLM\Data\market_cap"

market_value_dict = load_market_value_dict(tickers, folder)
print(market_value_dict)

{'AAPL': [1472619341.9127502, 1395791812.5103457, 1293344089.8828278, 1325355671.747543, 1363780453.2063904, 1447009831.6041412, 1517435623.2915802, 1581469803.778656, 1664700072.4194489, 1818366370.542664, 1843974990.6082304, 1799159599.4724426, 1747941914.2197876, 1767147349.9254456, 1728734364.186905, 1651894260.101532, 1581469803.778656, 1549458778.3158417, 1632688267.9939728, 1619884347.44252, 1562263477.829956, 1568665104.2645416, 1600678132.7742004, 1587872654.2974243, 1683906732.2092896, 1632688267.9939728, 1664700072.4194489, 1683906732.2092896, 1677503547.8493805, 1651894260.101532, 1639090895.9519806, 1587872654.2974243, 1530251228.282959, 1447009831.6041412, 1363780453.2063904, 1414998917.4217072, 1466217492.9174042, 1466217492.9174042, 1472619341.9127502, 1395791812.5103457, 1395791812.5103457, 1350975864.9726562, 1338159926.1401367, 1306148344.2754211, 1338159926.1401367, 1395791812.5103457, 1312550749.6726685, 1242125403.1067505, 1261332174.1769714, 1216516115.358902, 12

In [72]:
def evaluate_portfolio_performance_stable_position(
    position_dict: dict,
    market_value_dict: dict,
    price_data_dict: dict,
    initial_cash: float = 10000.0
):
    """
    Evaluate portfolio performance where stocks are only bought/sold when position changes (no daily rebalancing).

    Args:
        position_dict (dict): {ticker: [0/1, 0/1, ...]} for each day
        market_value_dict (dict): {ticker: [mv1, mv2, ...]} for each day
        price_data_dict (dict): {ticker: pd.DataFrame with 'Open' and 'Close', index is datetime}
        initial_cash (float): Starting cash

    Returns:
        dict with:
            - daily_values: list of portfolio values per day
            - CR: cumulative return
            - AR: annualized return
    """
    tickers = position_dict.keys()
    n_days = len(next(iter(position_dict.values())))
    cash = initial_cash
    holdings = {ticker: 0.0 for ticker in tickers}
    prev_positions = {ticker: 0 for ticker in tickers}
    daily_values = []

    for day_idx in range(n_days):
        # 1. Sell only positions that change from 1 to 0
        for ticker in tickers:
            prev = prev_positions[ticker]
            curr = position_dict[ticker][day_idx]
            if prev == 1 and curr == 0:
                open_price = float(price_data_dict[ticker].iloc[day_idx]['Open'])
                cash += holdings[ticker] * open_price
                holdings[ticker] = 0.0

        # 2. Identify newly added positions (0 to 1)
        new_longs = [t for t in tickers if prev_positions[t] == 0 and position_dict[t][day_idx] == 1]
        if new_longs:
            total_mv = sum(market_value_dict[t][day_idx] for t in new_longs)
            for ticker in new_longs:
                weight = market_value_dict[ticker][day_idx] / total_mv
                allocation = cash * weight
                open_price = float(price_data_dict[ticker].iloc[day_idx]['Open'])
                shares = allocation / open_price
                holdings[ticker] = shares
            cash = 0.0  # invest all cash

        # 3. Estimate portfolio value at day's close
        total_value = cash
        for ticker in tickers:
            if holdings[ticker] > 0:
                close_price = float(price_data_dict[ticker].iloc[day_idx]['Close'])
                total_value += holdings[ticker] * close_price
        daily_values.append(total_value)

        # 4. Update prev_positions
        for ticker in tickers:
            prev_positions[ticker] = position_dict[ticker][day_idx]
        print(holdings)

    # 5. Compute CR and AR
    CR = daily_values[-1] / initial_cash - 1
    AR = (1 + CR) ** (252 / n_days) - 1
    print("daily_values:", daily_values)
    return {
        "daily_values": daily_values,
        "CR": CR,
        "AR": AR
    }



In [7]:
import os
import pandas as pd

def load_market_value_dict(ticker_list, folder_path, start="1900-01-01", end="2100-01-01"):
    """
    Load MarketCap values for selected tickers within a date range.

    Args:
        ticker_list (list): List of tickers, e.g., ["AAPL", "TSLA"]
        folder_path (str): Path to folder with CSV files
        start (str): Start date in "YYYY-MM-DD"
        end (str): End date in "YYYY-MM-DD"

    Returns:
        dict: {ticker: [market_cap1, market_cap2, ...]} within date range
    """
    market_value_dict = {}

    for ticker in ticker_list:
        file_path = os.path.join(folder_path, f"{ticker}.csv")
        try:
            df = pd.read_csv(file_path)
            if 'Date' not in df.columns or 'MarketCap' not in df.columns:
                print(f"Warning: Missing required columns in {ticker}.csv")
                continue

            # 转换Date列为datetime格式，并过滤时间区间
            df['Date'] = pd.to_datetime(df['Date'])
            df_filtered = df[(df['Date'] >= start) & (df['Date'] <= end)]

            # 提取MarketCap序列
            market_value_dict[ticker] = df_filtered['MarketCap'].tolist()

        except FileNotFoundError:
            print(f"File not found: {ticker}.csv")
        except Exception as e:
            print(f"Error processing {ticker}.csv: {e}")

    return market_value_dict
tickers = ["AAPL", "TSLA"]
folder = r"D:\shawn_workspace\REAL LAB\REALLAB_FinLLM\Data\market_cap"

market_value_dict = load_market_value_dict(
    tickers,
    folder_path=folder,
    start="2024-01-01",
    end="2024-01-03"
)
print(market_value_dict)

{'AAPL': [2752524827601.4688, 2731914946253.25], 'TSLA': [800150877302.2461, 768037902170.4102]}


In [103]:
tsla_price_df = yf.download("TSLA", start="2024-01-01", end="2024-03-29")
print(tsla_price_df)

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            TSLA        TSLA        TSLA        TSLA       TSLA
Date                                                                 
2024-01-02  248.419998  251.250000  244.410004  250.080002  104654200
2024-01-03  238.449997  245.679993  236.320007  244.979996  121082600
2024-01-04  237.929993  242.699997  237.729996  239.250000  102629300
2024-01-05  237.490005  240.119995  234.899994  236.860001   92379400
2024-01-08  240.449997  241.250000  235.300003  236.139999   85166600
...                ...         ...         ...         ...        ...
2024-03-22  170.830002  171.199997  166.300003  166.690002   75454700
2024-03-25  172.630005  175.240005  168.729996  168.759995   74228600
2024-03-26  177.669998  184.250000  177.380005  178.580002  113186200
2024-03-27  179.830002  181.910004  176.000000  181.410004   81804000
2024-03-28  175.789993  179.570007  175.300003  177.449997   77654800

[61 rows x 5 column

In [104]:
dates = tsla_price_df.index
print(dates)

DatetimeIndex(['2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05',
               '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11',
               '2024-01-12', '2024-01-16', '2024-01-17', '2024-01-18',
               '2024-01-19', '2024-01-22', '2024-01-23', '2024-01-24',
               '2024-01-25', '2024-01-26', '2024-01-29', '2024-01-30',
               '2024-01-31', '2024-02-01', '2024-02-02', '2024-02-05',
               '2024-02-06', '2024-02-07', '2024-02-08', '2024-02-09',
               '2024-02-12', '2024-02-13', '2024-02-14', '2024-02-15',
               '2024-02-16', '2024-02-20', '2024-02-21', '2024-02-22',
               '2024-02-23', '2024-02-26', '2024-02-27', '2024-02-28',
               '2024-02-29', '2024-03-01', '2024-03-04', '2024-03-05',
               '2024-03-06', '2024-03-07', '2024-03-08', '2024-03-11',
               '2024-03-12', '2024-03-13', '2024-03-14', '2024-03-15',
               '2024-03-18', '2024-03-19', '2024-03-20', '2024-03-21',
      

In [33]:
start = "2024-01-01"
end = "2024-03-29"    
price_data = load_local_price_data(
    tickers=["AMZN"],
    start=start,
    end=end)

df = price_data["AMZN"]
df = df.loc[start:end]  # 自动按 index 过滤
print(df)
trading_days = df.index.strftime("%Y-%m-%d").tolist()
print(len(trading_days))


                           Open        High         Low       Close     Volume
Date                                                                          
2024-01-02 05:00:00  151.539993  152.380005  148.389999  149.929993   47339400
2024-01-03 05:00:00  149.199997  151.050003  148.330002  148.470001   49425500
2024-01-04 05:00:00  145.589996  147.380005  144.050003  144.570007   56039800
2024-01-05 05:00:00  144.690002  146.589996  144.529999  145.240005   45124800
2024-01-08 05:00:00  146.740005  149.399994  146.149994  149.100006   46757100
2024-01-09 05:00:00  148.330002  151.710007  148.210007  151.369995   43812600
2024-01-10 05:00:00  152.059998  154.419998  151.880005  153.729996   44421800
2024-01-11 05:00:00  155.039993  157.169998  153.119995  155.179993   49072700
2024-01-12 05:00:00  155.389999  156.199997  154.009995  154.619995   40460300
2024-01-16 05:00:00  153.529999  154.990005  152.149994  153.160004   41384600
2024-01-17 05:00:00  151.490005  152.149994  149.910

C:\Users\Spencer\AppData\Local\Temp\ipykernel_31944\2473696500.py:26: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")


In [30]:
from utils.fin_utils import generate_trading_days
start = "2024-01-01"
end = "2024-03-29"
trading_days = generate_trading_days(start, end)
print(trading_days)

['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25', '2024-01-26', '2024-01-29', '2024-01-30', '2024-01-31', '2024-02-01', '2024-02-02', '2024-02-05', '2024-02-06', '2024-02-07', '2024-02-08', '2024-02-09', '2024-02-12', '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16', '2024-02-19', '2024-02-20', '2024-02-21', '2024-02-22', '2024-02-23', '2024-02-26', '2024-02-27', '2024-02-28', '2024-02-29', '2024-03-01', '2024-03-04', '2024-03-05', '2024-03-06', '2024-03-07', '2024-03-08', '2024-03-11', '2024-03-12', '2024-03-13', '2024-03-14', '2024-03-15', '2024-03-18', '2024-03-19', '2024-03-20', '2024-03-21', '2024-03-22', '2024-03-25', '2024-03-26', '2024-03-27', '2024-03-28', '2024-03-29']


In [31]:
print(len(trading_days))

65


# amazon eval

In [ ]:
AMZNposition_dict:  

In [44]:
tickers = ["AMZN"]
folder = r"D:\shawn_workspace\REAL LAB\REALLAB_FinLLM\Data\market_cap"

AMZNmarket_value_dict = load_market_value_dict(
    tickers,
    folder_path=folder,
    start="2024-01-01",
    end="2024-03-29"
)
print(AMZNmarket_value_dict)

AMZNprice_data = load_local_price_data(
    tickers=["AMZN"],
    start="2024-01-01",
    end="2024-03-29"
)



result = evaluate_portfolio_performance(
    position_dict={'AMZN': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
    market_value_dict=AMZNmarket_value_dict,
    price_data_dict=AMZNprice_data
    # {
    #     "TSLA": tsla_price_df,  # 包含 'Open' 和 'Close' 列，index为日期
    #     "AMZN": amzn_price_df
    # }
)
print("")
print("Cumulative Return (CR):", result["CR"])
print("Annualized Return (AR):", result["AR"])
#

{'AMZN': [1591716755052.125, 1576216901955.3125, 1534813007251.875, 1541925975726.9062, 1582905285712.5625, 1607004396786.75, 1632059106964.4062, 1647452854380.125, 1641507696370.75, 1626007843273.9375, 1610614095858.2188, 1629617380352.0, 1649151517238.0625, 1643206359228.6875, 1656370753387.5938, 1665394596082.75, 1674737079808.0, 1689281495794.75, 1712000585041.0938, 1688007579648.0, 1647665227735.7188, 1690980158652.6875, 1824003636089.375, 1808079036281.375, 1795763973551.4375, 1810414657212.6875, 1803089315382.0625, 1852030925271.7188, 1829630315062.0625, 1790349667934.3438, 1815192004756.4065, 1802664730664.2808, 1799585883985.0938, 1773788110052.9688, 1789818815542.0625, 1853411109092.9688, 1857763871918.9065, 1855003504276.4065, 1842369962509.7808, 1838335840713.9373, 1876554783057.0935, 1892054798147.3125, 1885260308708.9688, 1848527493874.75, 1842051483473.0935, 1877191903123.875, 1861585782352.5625, 1825596193266.2188, 1862010367070.344, 1874431535481.375, 1897681477120.0, 

IndexError: single positional indexer is out-of-bounds

In [45]:
print(len([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))
print(len(AMZNmarket_value_dict["AMZN"]))
print(len(AMZNprice_data["AMZN"]))

65
61
61


In [29]:
print(AMZNmarket_value_dict["AMZN"])

[1591716755052.125, 1576216901955.3125, 1534813007251.875, 1541925975726.9062, 1582905285712.5625, 1607004396786.75, 1632059106964.4062, 1647452854380.125, 1641507696370.75, 1626007843273.9375, 1610614095858.2188, 1629617380352.0, 1649151517238.0625, 1643206359228.6875, 1656370753387.5938, 1665394596082.75, 1674737079808.0, 1689281495794.75, 1712000585041.0938, 1688007579648.0, 1647665227735.7188, 1690980158652.6875, 1824003636089.375, 1808079036281.375, 1795763973551.4375, 1810414657212.6875, 1803089315382.0625, 1852030925271.7188, 1829630315062.0625, 1790349667934.3438, 1815192004756.4065, 1802664730664.2808, 1799585883985.0938, 1773788110052.9688, 1789818815542.0625, 1853411109092.9688, 1857763871918.9065, 1855003504276.4065, 1842369962509.7808, 1838335840713.9373, 1876554783057.0935, 1892054798147.3125, 1885260308708.9688, 1848527493874.75, 1842051483473.0935, 1877191903123.875, 1861585782352.5625, 1825596193266.2188, 1862010367070.344, 1874431535481.375, 1897681477120.0, 185171244

In [51]:
start="2024-01-01"
end="2024-03-29"

def generate_trading_days(ticker, start, end):
    """
    提取某只股票在指定时间区间内的交易日（开市日）列表。

    参数：
        price_data_dict (dict): 如 {'AAPL': pd.DataFrame,...}，索引为datetime
        ticker (str): 股票代码，如 "AAPL"
        start (str): 开始日期，格式为 "YYYY-MM-DD"
        end (str): 结束日期，格式为 "YYYY-MM-DD"

    返回：
        List[str]: 交易日组成的日期字符串列表，格式为 "YYYY-MM-DD"
    """
    price_data = load_local_price_data(
    tickers=[ticker],
    start=start,
    end=end)

    df = price_data[ticker]
    df = df.loc[start:end]  # 自动按 index 过滤
    trading_days = df.index.strftime("%Y-%m-%d").tolist()
    return trading_days

td = generate_trading_days("TSLA", start, end)

In [ ]:
import re

def sanitize_and_validate_time(t: str) -> str:
    t_clean = t.replace("-", "")
    if not re.match(r"^\d{8}T\d{4}$", t_clean):
        raise ValueError(f"Invalid time format: '{t}'. Expected 'YYYYMMDDTHHMM', e.g., '20240101T0900'")
    return t_clean


time_from = sanitize_and_validate_time('2023/01/01T0000')


print(time_from)


20230101T0000


In [ ]:
2025-05-26 17:36:05,140 | INFO | Running portfolio simulation
2025-05-26 17:36:05,141 | INFO | Tickers: ['AAPL']
2025-05-26 17:36:05,141 | INFO | Start date: 2024-01-01
2025-05-26 17:36:05,141 | INFO | End date: 2024-03-29
2025-05-26 17:36:05,141 | INFO | Cumulative Return (CR): -0.053649092710523316
2025-05-26 17:36:05,141 | INFO | Annualized Return (AR): -0.2037157667385986
position_dict: {'AAPL': [1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0]}
Cumulative Return (CR): -0.053649092710523316
Annualized Return (AR): -0.2037157667385986

61


In [6]:
# January 1st, 2024, to March 29th, 2024
# tsla_price_df = yf.download("TSLA", start="2024-01-25", end="2024-02-02")
# amzn_price_df = yf.download("AMZN", start="2024-01-25", end="2024-02-02")
price_df = load_local_price_data(
    tickers=["TSLA", "AMZN"],
    start="2024-01-25",
    end="2024-02-02"
)

# print(tsla_price_df)
# print(amzn_price_df)
result = evaluate_portfolio_performance(
    position_dict={
        "TSLA": [1, 1, 0, 1, 0],
        "AMZN": [1, 1, 1, 1, 0]
    },
    market_value_dict={
        "TSLA": [2, 2, 4, 6, 3],
        "AMZN": [1, 1, 3, 4, 6]
    },
    price_data_dict=price_df
    # {
    #     "TSLA": tsla_price_df,  # 包含 'Open' 和 'Close' 列，index为日期
    #     "AMZN": amzn_price_df
    # }
)
print("")
print("Cumulative Return (CR):", result["CR"])
print("Annualized Return (AR):", result["AR"])

# print("\n")
# result = evaluate_portfolio_performance_stable_position(
#     position_dict={
#         "TSLA": [0, 1, 0, 1, 0],
#         "AMZN": [1, 1, 1, 1, 0]
#     },
#     market_value_dict={
#         "TSLA": [2, 3, 4, 6, 3],
#         "AMZN": [1, 2, 3, 4, 6]
#     },
#     price_data_dict={
#         "TSLA": tsla_price_df,  # 包含 'Open' 和 'Close' 列，index为日期
#         "AMZN": amzn_price_df
#     }
# )
# print("")
# print("Cumulative Return (CR):", result["CR"])
# print("Annualized Return (AR):", result["AR"])




new day
有变动
变动前 卖掉所有  {'TSLA': 0.0, 'AMZN': 0.0}
cash:  10000.0
有股票要买
TSLA  weight  0.6666666666666666  allocation  6666.666666666666  shares 35.143209140301686
AMZN  weight  0.3333333333333333  allocation  3333.333333333333  shares 21.238186671851768
买完股票  {'TSLA': 35.143209140301686, 'AMZN': 21.238186671851768}
{'TSLA': 35.143209140301686, 'AMZN': 21.238186671851768}

new day
cash:  0.0
{'TSLA': 35.143209140301686, 'AMZN': 21.238186671851768}

new day
有变动
变动前 卖掉所有  {'TSLA': 0.0, 'AMZN': 0.0}
cash:  9907.7266708282
有股票要买
AMZN  weight  1.0  allocation  9907.7266708282  shares 62.179784727861005
买完股票  {'TSLA': 0.0, 'AMZN': 62.179784727861005}
{'TSLA': 0.0, 'AMZN': 62.179784727861005}

new day
有变动
变动前 卖掉所有  {'TSLA': 0.0, 'AMZN': 0.0}
cash:  9992.29121600962
有股票要买
TSLA  weight  0.6  allocation  5995.374729605772  shares 30.693568184120053
AMZN  weight  0.4  allocation  3996.9164864038485  shares 24.871913891144406
买完股票  {'TSLA': 30.693568184120053, 'AMZN': 24.871913891144406}
{'TSLA': 30

C:\Users\Spencer\AppData\Local\Temp\ipykernel_22960\2473696500.py:26: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
C:\Users\Spencer\AppData\Local\Temp\ipykernel_22960\2473696500.py:26: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")


In [94]:
tsla_price_df = yf.download("TSLA", start="2024-01-25", end="2024-01-30")
amzn_price_df = yf.download("AMZN", start="2024-01-25", end="2024-01-30")
print(tsla_price_df)
result = evaluate_portfolio_performance(
    position_dict={
        "TSLA": [0, 1, 1],
        "AMZN": [1, 0, 1]
    },
    market_value_dict={
        "TSLA": [2, 3, 4],
        "AMZN": [1, 2, 3]
    },
    price_data_dict={
        "TSLA": tsla_price_df,  # 包含 'Open' 和 'Close' 列，index为日期
        "AMZN": amzn_price_df
    }
)

print("Cumulative Return (CR):", result["CR"])
print("Annualized Return (AR):", result["AR"])

result = evaluate_portfolio_performance_stable_position(
    position_dict={
        "TSLA": [0, 1, 1],
        "AMZN": [1, 0, 1]
    },
    market_value_dict={
        "TSLA": [2, 3, 4],
        "AMZN": [1, 2, 3]
    },
    price_data_dict={
        "TSLA": tsla_price_df,  # 包含 'Open' 和 'Close' 列，index为日期
        "AMZN": amzn_price_df
    }
)

print("Cumulative Return (CR):", result["CR"])
print("Annualized Return (AR):", result["AR"])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            TSLA        TSLA        TSLA        TSLA       TSLA
Date                                                                 
2024-01-25  182.630005  193.000000  180.059998  189.699997  198076800
2024-01-26  183.250000  186.779999  182.100006  185.500000  107343200
2024-01-29  190.929993  191.479996  183.669998  185.630005  125013100
cash:  10000.0
AMZN  weight  1.0  allocation  10000.0  shares 63.714560015555314
{'TSLA': 0.0, 'AMZN': 63.714560015555314}
cash:  10093.66048099943
TSLA  weight  1.0  allocation  10093.66048099943  shares 54.41326404851444
{'TSLA': 54.41326404851444, 'AMZN': 0.0}
cash:  10100.734471015501
TSLA  weight  0.5714285714285714  allocation  5771.848269151715  shares 31.09329374200825
AMZN  weight  0.42857142857142855  allocation  4328.886201863786  shares 27.167605757214318
{'TSLA': 31.09329374200825, 'AMZN': 27.167605757214318}
daily_values: [10050.971842453851, 9971.230636890272


C:\Users\Spencer\AppData\Local\Temp\ipykernel_1256\1358167747.py:52: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  open_price = float(price_data_dict[ticker].iloc[day_idx]['Open'])
C:\Users\Spencer\AppData\Local\Temp\ipykernel_1256\1358167747.py:62: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  close_price = float(price_data_dict[ticker].iloc[day_idx]['Close'])
C:\Users\Spencer\AppData\Local\Temp\ipykernel_1256\1358167747.py:38: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  open_price = float(price_data_dict[ticker].iloc[day_idx]['Open'])
C:\Users\Spencer\AppData\Local\Temp\ipykernel_1256\3167637019.py:46: FutureWarning: Calling float on a single element Series is deprecated and will raise a T